In [ ]:
#api_key = ""

# Cài đặt thư viện cần thiết

In [2]:
!pip install --quiet -U langchain chromadb langchain-openai pypdf gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.3 MB/s eta 0:

In [5]:
!pip install --quiet -U langchain chromadb langchain-openai pypdf gradio langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00


# Xử lí văn bản PDF

In [9]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
loader = PyPDFLoader("/content/kichbanchatbot-đã gộp.pdf")
splits = loader.load_and_split(text_splitter)

# Đưa văn bản thành Embeddings và Lưu trữ vào vectorstore

In [ ]:
#emb = OpenAIEmbeddings(openai_api_key=api_key)
vectorstore = Chroma.from_documents(documents=splits, embedding=emb)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-8BHC9***************************************nx2j. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

# Xây Dựng Gradio

In [ ]:
import gradio as gr
from openai import OpenAI

#llm = OpenAI(api_key=api_key)

RAG_PROMPT = """Nhiệm vụ của bạn là trả lời câu hỏi của người dùng dựa trên dữ liệu được cho.
Nếu dữ liệu được cho không liên quan đến câu hỏi, vui lòng trả lời "Tôi không biết"
---
Dữ liệu: {context}
---
Câu hỏi: {question}
---
Trả lời:"""

def predict(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human })
        history_openai_format.append({"role": "assistant", "content":assistant})
    docs = vectorstore.similarity_search(message)
    context = docs[0].page_content
    history_openai_format.append({"role": "user", "content": RAG_PROMPT.format(context=context, question=message)})

    response = llm.chat.completions.create(model='gpt-3.5-turbo',
      messages= history_openai_format,
      temperature=1.0,
      stream=True)

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
              partial_message = partial_message + chunk.choices[0].delta.content
              yield partial_message



gr.ChatInterface(predict).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bf5b90d5d430da2480.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ReadTimeout: The read operation timed out